In [15]:
import psycopg2
import pandas as pd

Connection with DB

In [16]:
import psycopg2

connect = psycopg2.connect(
    host = "localhost",
    user = "root",
    password = "root",
    database = "postgres",
    port = "5432"

)
connect.autocommit = True

SQL query to create the table

In [17]:
def makeQuery():
    cursor = connect.cursor()

    # SQL query to create the table
    query = """
    CREATE TABLE IF NOT EXISTS applicants (
        first_name VARCHAR(500),
        last_name VARCHAR(500),
        email VARCHAR(500),
        application_date VARCHAR(500),
        country VARCHAR(500),
        years_of_experience VARCHAR(500),
        seniority VARCHAR(500),
        technology VARCHAR(500),
        code_challenge_score VARCHAR(500),
        technical_interview_score VARCHAR(500)
    );
    """

    cursor.execute(query)
    cursor.close()

makeQuery()

# Insert the data from candidates.csv uploaded in the storage of Pgadmin.

query = """
COPY applicants (first_name, last_name, email, application_date, country, years_of_experience, seniority, technology, code_challenge_score, technical_interview_score) 
FROM 'D:\Workshop1david\data\pgadmin-data\storage\davidvelasquezlenis_gmail.com\candidates.csv' DELIMITER ';' CSV HEADER;
"""
makeQuery()

EDA

Make the pandas Dataframe

In [18]:
cursor = connect.cursor()
cursor.execute("SELECT * FROM applicants")
dbtable = cursor.fetchall()
df = pd.DataFrame(dbtable)

#1 Data Understanding:
Displays the first rows of data, displays information about the DataFrame, and displays statistical summary of the DataFrame

In [19]:
print(df.head())  # Muestra las primeras filas de datos
print(df.info())  # Muestra información sobre el DataFrame
print(df.describe())  # Resumen estadístico del DataFrame
print(df.dtypes) # Tipos de dato por columnas
df.isna().sum() # Suma de nulos por columnas

            0          1                         2                 3        4  \
0  First Name  Last Name                     Email  Application Date  Country   
1  Bernadette  Langworth       leonard91@yahoo.com        2021-02-26   Norway   
2      Camryn   Reynolds       zelda56@hotmail.com        2021-09-09   Panama   
3       Larue     Spinka  okey_schultz41@gmail.com        2020-04-14  Belarus   
4        Arch     Spinka    elvera_kulas@yahoo.com        2020-10-01  Eritrea   

     5          6               7                     8  \
0  YOE  Seniority      Technology  Code Challenge Score   
1    2     Intern   Data Engineer                     3   
2   10     Intern   Data Engineer                     2   
3    4  Mid-Level  Client Success                    10   
4   25    Trainee       QA Manual                     7   

                           9  
0  Technical Interview Score  
1                          3  
2                         10  
3                          9  
4  

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

In [20]:
#añadir graficos

#2 Data preparation:
First of all, change the hearders to the original names, not numbers.

In [21]:
# Extract the second row as column headers
new_headers = df.iloc[0]

# Assign the new headers to the DataFrame
df.columns = new_headers

# Drop the second row
df = df.drop(df.index[0])

print(df.head())

0  First Name   Last Name                      Email Application Date  \
1  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
2      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
3       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
4        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
5       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

0  Country YOE  Seniority                         Technology  \
1   Norway   2     Intern                      Data Engineer   
2   Panama  10     Intern                      Data Engineer   
3  Belarus   4  Mid-Level                     Client Success   
4  Eritrea  25    Trainee                          QA Manual   
5  Myanmar  13  Mid-Level  Social Media Community Management   

0 Code Challenge Score Technical Interview Score  
1                    3                         3  
2                    2                        10  
3                   10 

In [26]:
# Normalizar los nombres de las columnas INGLES
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [27]:
# Reiniciar el índice y agregar una columna 'id'
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)

I change the data type of some columns that are numeric and were as object

In [29]:
df['yoe'] = df['yoe'].astype(int)
df['code_challenge_score'] = df['code_challenge_score'].astype(int)
df['technical_interview_score'] = df['technical_interview_score'].astype(int)
df['application_date'] = pd.to_datetime(df['application_date'])
print(df.dtypes)

0
id                                    int64
first_name                           object
last_name                            object
email                                object
application_date             datetime64[ns]
country                              object
yoe                                   int32
seniority                            object
technology                           object
code_challenge_score                  int32
technical_interview_score             int32
year                                  int64
month                                 int64
day                                   int64
dtype: object


I divide the Application Date column into three other new columns: Year, month and day.

In [31]:
df['year'] = df['application_date'].dt.year
df['month'] = df['application_date'].dt.month
df['day'] = df['application_date'].dt.day
print(df['year'], df['month'], df['day'])

0        2021
1        2021
2        2020
3        2020
4        2020
         ... 
49995    2022
49996    2020
49997    2018
49998    2020
49999    2022
Name: year, Length: 50000, dtype: int64 0         2
1         9
2         4
3        10
4         5
         ..
49995     1
49996     6
49997    12
49998     5
49999     6
Name: month, Length: 50000, dtype: int64 0        26
1         9
2        14
3         1
4        20
         ..
49995     9
49996     2
49997    15
49998    30
49999    13
Name: day, Length: 50000, dtype: int64
